# Modelo de reconocimiento de imagenes satelitales con CNN
### Santiago Fandiño Gomez
### Juan Manuel Durán 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns 
from pathlib import Path

In [2]:
def split_dataset(source_folder, train_folder, test_folder, split_ratio=0.6):
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    files = os.listdir(source_folder)
    random.shuffle(files)
    num_train = int(len(files) * split_ratio)
    for file in files[:num_train]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(train_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de entrenamiento creada con exito")

    for file in files[num_train:]:
        src_file = os.path.join(source_folder, file)
        dst_file = os.path.join(test_folder, file)
        shutil.copy(src_file, dst_file)
    print("Carpeta de pruebas creada con exito")


In [3]:
def reorder(ruta_data):
    carpetas = os.listdir(ruta_data)
    carpeta_train = os.path.join(ruta_data, 'train')
    carpeta_test = os.path.join(ruta_data, 'test')
    os.makedirs(carpeta_train, exist_ok=True)
    os.makedirs(carpeta_test, exist_ok=True)
    for carpeta in carpetas:
        if os.path.isdir(os.path.join(ruta_data, carpeta)) and (carpeta.endswith('_train') or carpeta.endswith('_test')):
            carpeta_destino = carpeta_train if carpeta.endswith('_train') else carpeta_test
            archivos_a_mover = os.listdir(os.path.join(ruta_data, carpeta))
            for archivo_a_mover in archivos_a_mover:
                origen = os.path.join(ruta_data, carpeta, archivo_a_mover)
                destino = os.path.join(carpeta_destino, archivo_a_mover)
                shutil.move(origen, destino)
            os.rmdir(os.path.join(ruta_data, carpeta))
    
    # Una vez tenidas todas las imágenes en las carpetas correspondientes, separar cada una de las carpetas en las categorías 
    # cloudy, desert, green_area y water

    categories = ["cloudy", "desert", "green_area", "water"]
    for category in categories:
        os.makedirs(os.path.join(carpeta_train, category), exist_ok=True)
        os.makedirs(os.path.join(carpeta_test, category), exist_ok=True)

    archivos_t = os.listdir(carpeta_train)
    archivos_ts = os.listdir(carpeta_test)

    for archivo in archivos_t:
        for category in categories:
            if category in archivo:
                origen = os.path.join(carpeta_train, archivo)
                destino = os.path.join(carpeta_train, category)
                shutil.move(origen, destino)

    for archivo in archivos_ts:
        for category in categories:
            if category in archivo:
                origen = os.path.join(carpeta_test, archivo)
                destino = os.path.join(carpeta_test, category)
                shutil.move(origen, destino)


In [4]:
def rename(path, category):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            file_path = os.path.join(path, file)
            file_name, file_extension = os.path.splitext(file)
            new_file_name = f"{file_name}_{category}{file_extension}"
            os.rename(file_path, os.path.join(path, new_file_name))

In [5]:
def resize_images_in_folder(folder_name, target_size):
    
    datagen = ImageDataGenerator()
    datagen_config = {"directory": folder_name, "target_size": target_size}
    image_generator = datagen.flow_from_directory(**datagen_config, shuffle=False)
    
    for i in range(len(image_generator)):
        batch_images, batch_labels = image_generator[i]
        for j, image in enumerate(batch_images):
            if image.shape[:2] != target_size:
                resized_image = datagen.load_img(image_generator.filepaths[i * image_generator.batch_size + j], target_size=target_size)
                datagen.save_img(image_generator.filepaths[i * image_generator.batch_size + j], resized_image)

*Las dos funciones anteriores preparan las imagenes para ser trabajadas como prueba y testeo de acuerdo a los parametros que se le entreguen a las funciones. De igual manera, se trabaja una función para asignar un tamaño estandar a las imagenes de las carpetas que se especifique*

In [6]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print("GPU disponible para usar.")
else:
    print("GPU no disponible")

GPU no disponible


In [7]:
resize_images_in_folder("Data",(128,128))

Found 5631 images belonging to 4 classes.


*El proceso de re organización de las imagenes en carpetas se dificulta debido a los nombres, la funcion de rename se encarga de poner los nombres de la categoria y un numero adicional para facilitar eeste trabajo*

In [8]:
categories = ["cloudy", "desert", "green_area", "water"]
for category in categories:
    path = os.path.join("Data", category)
    rename(path, category)


*Teniendo todas las imagenes con el tamaño solicitado, queda realizar las divisiones de los dataset de imagenes en training y test*

In [9]:
folders = ["cloudy", "desert", "green_area", "water"]
split_ratio = 0.6

for folder in folders:
    source_folder = os.path.join("Data", folder)
    train_folder = os.path.join("Data", folder + "_train")
    test_folder = os.path.join("Data", folder + "_test")
    split_dataset(source_folder, train_folder, test_folder, split_ratio)

ruta_data = 'Data'
reorder(ruta_data)

print("carpetas de entrenamiento y pruebas creadas")

Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
Carpeta de entrenamiento creada con exito
Carpeta de pruebas creada con exito
carpetas de entrenamiento y pruebas creadas


*Teniendo las imagenes separadas en test y train se puede continuar con la definición de las redes CNN*

In [10]:
train_path = Path('Data/train/')
test_path = Path('Data/test/')

In [11]:
img_width, img_height = 128, 128
batch_size = 32

Las dimensiones anteriores son siguiendo las recomendaciones del profesor

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Rotacion entre -20 y 20 grados
    width_shift_range=0.1,  # Shift horizontal hasta del 10%
    height_shift_range=0.1, # Shift vertical hasta dele 10%
    shear_range=0.2,        
    zoom_range=0.2,         # Random zoom hasta del 20%
    horizontal_flip=True,   # voltear la imagen de manera horizontal
    fill_mode='nearest'     # Llenar los nuevos pixeles creados
)

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
if train_path.exists():
    archivos = os.listdir(train_path)
    print(f'Dentro de la carpeta {str(train_path)} se encuentran {len(archivos)} archivos')
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
    )
else:
    print('Ruta no existe')

Dentro de la carpeta Data\train se encuentran 4 archivos
Found 3378 images belonging to 4 classes.


In [15]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # mantener los datos en el mismo orden en que se tienen
)

Found 2253 images belonging to 4 classes.


*Teniendo los generadores de imagenes de entrenamiento y de imagenes de prueba, se puede iniciar con el desarrollo del modelo CNN*

In [16]:
inputs = Input(shape=(img_width, img_height,3))
# El objeto de tipo input tiene la altura y ancho definidos. 
# Al ser imagenes a color se cuenta con 3 canales
x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
# 32- numero de filtros aplicados
# (3,3) es el tamaño del kernel de convolusion 
# Se tienen en cuenta los recomendados en clase, trabajando valores inpares
# se aplica una capa convolusional con activación reul
# la misma trabajada en clases. Se le envian los inputs
x = BatchNormalization()(x)
# Ayuda a estabilizar y acelear el proceso de la red neuronal
x = MaxPooling2D((2, 2), strides=(2,2))(x)
# Reduce la dimensionalidad y hace la red mas robusta
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)
#Trabajando con softmax nos debera de clasificar entre dos clases
#nos dira una distribución de probabilidad entre estas dos
model = Model(inputs=inputs, outputs=outputs)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,837,572 (22.27 MB)

 Trainable params: 5,836,612 (22.26 MB)

 Non-trainable params: 960 (3.75 KB)

In [17]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10


c:\Users\juanm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


106/106 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.6955 - loss: 0.3982 - val_accuracy: 0.2663 - val_loss: 0.6451
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 345s 3s/step - accuracy: 0.9151 - loss: 0.1222 - val_accuracy: 0.2663 - val_loss: 0.6363
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 378s 4s/step - accuracy: 0.9473 - loss: 0.0805 - val_accuracy: 0.2663 - val_loss: 0.7018
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 357s 3s/step - accuracy: 0.9523 - loss: 0.0701 - val_accuracy: 0.3067 - val_loss: 0.7425
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - accuracy: 0.9603 - loss: 0.0583 - val_accuracy: 0.6906 - val_loss: 0.5333
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.9637 - loss: 0.0512 - val_accuracy: 0.8304 - val_loss: 0.2543
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.9616 - loss: 0.0582 - val_accuracy: 0.9458 - val_loss: 0.0828
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 283s 3s/step - accuracy: 0.9664 - loss: 0.0511 - val_accuracy: 0.986

In [18]:
# Obtener la precisión y pérdida en el conjunto de entrenamiento y de prueba
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [19]:
print("Training Loss:", train_loss[-1])
print("Training Accuracy:", train_accuracy[-1])
print("Validation Loss:", val_loss[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Loss: 0.0433267280459404
Training Accuracy: 0.9703966975212097
Validation Loss: 0.023956477642059326
Validation Accuracy: 0.9835774302482605


##Segundo modelo entrenado

- Reducción del número de filtros en las capas convolucionales: Comenzado con 16 filtros en la primera capa convolucional, y manteniendo este número bajo a lo largo del modelo.
- Reducción del tamaño de las capas densas: Reducido el número a 128 neuronas en la única capa densa que sigue a las capas convolucionales. 
- Reducción de la profundidad de la red: Se mantienen solo cinco capas convolucionales seguidas de una capa de reducción global de promedio y una capa densa.

In [20]:
inputs = Input(shape=(img_width, img_height,3))
x = Conv2D(16, (3, 3), activation='relu', padding="same")(inputs)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding="same")(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,020 (1.63 MB)

 Trainable params: 426,020 (1.63 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 36s 305ms/step - accuracy: 0.3236 - loss: 0.6828 - val_accuracy: 0.2663 - val_loss: 0.6442
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 31s 281ms/step - accuracy: 0.2702 - loss: 0.6181 - val_accuracy: 0.2663 - val_loss: 0.5549
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 32s 287ms/step - accuracy: 0.2691 - loss: 0.5464 - val_accuracy: 0.2663 - val_loss: 0.5163
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 32s 291ms/step - accuracy: 0.3891 - loss: 0.4951 - val_accuracy: 0.6622 - val_loss: 0.4166
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 31s 283ms/step - accuracy: 0.6652 - loss: 0.3927 - val_accuracy: 0.6813 - val_loss: 0.3337
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 31s 279ms/step - accuracy: 0.6834 - loss: 0.3188 - val_accuracy: 0.6840 - val_loss: 0.2927
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 32s 289ms/step - accuracy: 0.6963 - loss: 0.2830 - val_accuracy: 0.7248 - val_loss: 0.2717
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 33s 298ms/step - accuracy: 0.7238 - loss: 0

In [22]:
# Obtener la precisión y pérdida en el conjunto de entrenamiento y de prueba
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [23]:
print("Training Loss:", train_loss[-1])
print("Training Accuracy:", train_accuracy[-1])
print("Validation Loss:", val_loss[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Loss: 0.24588218331336975
Training Accuracy: 0.741563081741333
Validation Loss: 0.24959857761859894
Validation Accuracy: 0.6835330724716187


## Modelo 3
Reducción de capas de profundidad

In [24]:
inputs = Input(shape=(img_width, img_height, 3))
x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,172 (434.27 KB)

 Trainable params: 110,724 (432.52 KB)

 Non-trainable params: 448 (1.75 KB)

In [25]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 76s 692ms/step - accuracy: 0.4620 - loss: 0.5534 - val_accuracy: 0.2983 - val_loss: 0.6646
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 75s 696ms/step - accuracy: 0.8372 - loss: 0.3293 - val_accuracy: 0.5095 - val_loss: 0.5814
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 74s 685ms/step - accuracy: 0.8708 - loss: 0.2573 - val_accuracy: 0.6516 - val_loss: 0.4750
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 72s 671ms/step - accuracy: 0.8661 - loss: 0.2245 - val_accuracy: 0.7093 - val_loss: 0.3672
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 72s 669ms/step - accuracy: 0.8867 - loss: 0.1986 - val_accuracy: 0.8402 - val_loss: 0.2637
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 81s 750ms/step - accuracy: 0.8923 - loss: 0.1813 - val_accuracy: 0.8842 - val_loss: 0.2012
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 83s 771ms/step - accuracy: 0.8895 - loss: 0.1755 - val_accuracy: 0.9037 - val_loss: 0.1691
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 86s 796ms/step - accuracy: 0.8865 - loss: 0

In [26]:
# Obtener la precisión y pérdida en el conjunto de entrenamiento y de prueba
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [27]:
print("Training Loss:", train_loss[-1])
print("Training Accuracy:", train_accuracy[-1])
print("Validation Loss:", val_loss[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Loss: 0.15595239400863647
Training Accuracy: 0.9026051163673401
Validation Loss: 0.1447286158800125
Validation Accuracy: 0.9085663557052612


## Modelo 4
aumentar tamaño de filtros y aumentar profundidad

In [28]:
inputs = Input(shape=(img_width, img_height, 3))
x = Conv2D(64, (5, 5), activation='relu', padding="same")(inputs)  # Aumento del tamaño del filtro
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (5, 5), activation='relu', padding="same")(x)  # Aumento del tamaño del filtro y más filtros
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(256, (5, 5), activation='relu', padding="same")(x)  # Aumento del tamaño del filtro y más filtros
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(512, (5, 5), activation='relu', padding="same")(x)  # Aumento del tamaño del filtro y más filtros
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Más neuronas en la capa densa
outputs = Dense(4, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 128, 128, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 64, 64, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 32, 32, 256)    │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 16, 16, 512)    │     3,277,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,442,756 (16.95 MB)

 Trainable params: 4,440,836 (16.94 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [29]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 463s 4s/step - accuracy: 0.7868 - loss: 0.3171 - val_accuracy: 0.4576 - val_loss: 0.6020
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 459s 4s/step - accuracy: 0.9228 - loss: 0.1132 - val_accuracy: 0.5260 - val_loss: 0.6666
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 548s 5s/step - accuracy: 0.9298 - loss: 0.1004 - val_accuracy: 0.5078 - val_loss: 0.8061
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 616s 6s/step - accuracy: 0.9451 - loss: 0.0887 - val_accuracy: 0.5291 - val_loss: 0.8383
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 490s 5s/step - accuracy: 0.9641 - loss: 0.0665 - val_accuracy: 0.6059 - val_loss: 0.4885
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 468s 4s/step - accuracy: 0.9653 - loss: 0.0613 - val_accuracy: 0.7967 - val_loss: 0.2444
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 433s 4s/step - accuracy: 0.9598 - loss: 0.0595 - val_accuracy: 0.8726 - val_loss: 0.1841
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 416s 4s/step - accuracy: 0.9621 - loss: 0.0586 - val_accu

In [30]:
# Obtener la precisión y pérdida en el conjunto de entrenamiento y de prueba
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [31]:
print("Training Loss:", train_loss[-1])
print("Training Accuracy:", train_accuracy[-1])
print("Validation Loss:", val_loss[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Loss: 0.050716083496809006
Training Accuracy: 0.9668442606925964
Validation Loss: 0.07988697290420532
Validation Accuracy: 0.9347536563873291
